## 读取问答数据集

In [2]:
import json
import pdfplumber

questions = json.load(open("questions.json"))
print(questions[0])

{'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '', 'reference': ''}


In [4]:
pdf = pdfplumber.open("trainData.pdf")
print(len(pdf.pages)) # 页数
pdf.pages[0].extract_text() # 读取第一页内容

354


'欢迎\n感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。\n首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。\n本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际\n接收的车辆为准。\n如您有任何问题，或需要预约服务，请拨打电话4006-010101联系我们。您也可以开车前往Lynk&Co领克中心。\n在抵达之前，请您注意驾车安全。\n©领克汽车销售有限公司'

## 读取pdf 所有页内容

In [5]:
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        "page": 'page_' + str(page_idx + 1),
        "content": pdf.pages[page_idx].extract_text()
    })

In [8]:
pdf_content[3]

{'page': 'page_4',
 'content': '目录\n组合仪表................................................................69 前排座椅通风...........................................................115\n指示灯和警告灯........................................................69 前排座椅通风...........................................................117\n指示灯和警告灯........................................................74 头枕.....................................................................118\n查看组合仪表信息......................................................78 方向盘加热.............................................................119\n查看组合仪表信息.....................................................80 方向盘加热............................................................120\n打开/关闭远近光灯....................................................82 儿童锁...................................................................121\n打开/关闭远近光灯....................................................83 儿童座椅固定装置....................................................122\n打开/关闭自动

### TFIDF 对文档检索进行预处理

In [19]:
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

In [20]:
# 对提问和pdf内容进行分词
question_words = [" ".join(jieba.lcut(x['question'])) for x in questions]
pdf_content_words = [" ".join(jieba.lcut(x['content'])) for x in pdf_content]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.966 seconds.
Prefix dict has been built successfully.


In [24]:
pdf_content_words[3]

'目录 \n 组合 仪表 ................................................................ 69   前排 座椅 通风 ........................................................... 115 \n 指示灯 和 警告 灯 ........................................................ 69   前排 座椅 通风 ........................................................... 117 \n 指示灯 和 警告 灯 ........................................................ 74   头枕 ..................................................................... 118 \n 查看 组合 仪表 信息 ...................................................... 78   方向盘 加热 ............................................................. 119 \n 查看 组合 仪表 信息 ..................................................... 80   方向盘 加热 ............................................................ 120 \n 打开 / 关闭 远 近光灯 .................................................... 82   儿童 锁 ................................................................... 121 \n 打开 / 关闭 远 近光灯 .................................................... 83   儿童座椅 固定装置 ...........

In [12]:
tfidf = TfidfVectorizer()
tfidf.fit(question_words + pdf_content_words)

# 提取TFIDF
question_feat = tfidf.transform(question_words)
pdf_content_feat = tfidf.transform(pdf_content_words)

# 进行归一化
question_feat = normalize(question_feat) # (301, 4568) 每一个问题被映射成为了一个4568的向量
pdf_content_feat = normalize(pdf_content_feat) # (354, 4568) 每一页也被映射成为了4568的向量

In [16]:
question_feat.shape, pdf_content_feat.shape

((301, 4568), (354, 4568))

In [19]:
# 检索进行排序

for query_idx, feat in enumerate(question_feat):
    score = feat @ pdf_content_feat.T # 1 4568 * 4568 354 # 余弦相似度 只不过标准化了模为1 与每个文档的句子的余弦相似度的句矩阵
    score = score.toarray()[0] # 1 354 -> 354
    max_score_page_idx = score.argsort()[-1] + 1 # argsort函数返回的是数组值从小到大的索引值
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)
 


In [20]:
# 生成提交结果 这里的生成结果只是定位到几页 但是没有抽取关键词因此得分是0
with open('submit.json', 'w', encoding='utf-8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)


## BM25
![BM25intro](img/BM25.png)

!pip install rank_bm25


In [23]:
!pip install rank_bm25

In [24]:
from rank_bm25 import BM25Okapi
pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]

bm25 = BM25Okapi(pdf_content_words)

for query_idx in range(len(questions)):
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]['question']))
    max_score_page_idx = doc_scores.argsort()[-1] + 1
    questions[query_idx]['reference'] = "page_" + str(max_score_page_idx)

with open('submitBM25.json', 'w', encoding='utf-8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

### 文本嵌入与向量搜索
目的：对文本进行编码，并进行语义检索

#### 语义检索流程
语义检索是指通过词嵌入和句子嵌入等技术，将文本表示为语义丰富的向量。通过相似度计算和结果排序找到最相关的文档。用户查询经过自然语言处理处理，最终系统返回经过排序的相关文档，提供用户友好的信息展示。一个简单的流程图如下
![语义检索](img/semantic_retriveal.png)


##### 文本编码模型
大多数语义检索系统采用预训练模型进行文本编码，其中最为常见的是基于BERT（Bidirectional Encoder Representations from Transformers）的模型，或者使用GPT（Generative Pre-trained Transformer）等。这些预训练模型通过在大规模语料上进行训练，能够捕捉词语和句子之间的复杂语义关系。选择合适的文本编码模型直接影响到得到的文本向量的有效性，进而影响检索的准确性和效果。

In [29]:
# M3E
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('moka-ai/m3e-small')

.gitattributes:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/95.8M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\Lenovo\miniconda3\envs\LLMDev\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [30]:
question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]

question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)

# ((301, 512), (354, 512))

In [39]:
pdf_content_sentences = [x['content'].replace("..", '') for x in pdf_content]
count = 0
chars = 0
for text in pdf_content_sentences:
    if len(text) > 100:
        count += 1
        chars += len(text)

avg_token = chars/count
avg_token

429.8774834437086

In [32]:
for query_idx, feat in enumerate(question_embeddings):
    score = feat @ pdf_embeddings.T
    max_score_page_idx = score.argsort()[-1] + 1
    questions[query_idx]['reference'] = "page_" + str(max_score_page_idx)


with open('submitM3e.json', "w", encoding='utf-8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

##### 文本切分[并没有实现，是一个待改进的点] 还有就是不是有目录吗，可以通过问题，解析目录，然后定位内容吗。这个好像从技术上很难做到，
而且就算定位到了那标记的一页，你还是需要通过TFIDF等方法来判断相似性，so,it may not make sense
reference:
- https://python.langchain.com/docs/modules/data_connection/document_transformers/
- 

文本的长度会影响文本编码的结果。短文本和长文本在编码成向量时可能
表达不同的语义信息。即使两者包含相同的单词或有相似的语义，由于上下文的不同，得到的向量也会有所不同。因此，当在语义检索中使用短文本来检索长文本时，或者反之，可能导致一定的误差。针对文本长度的差异，有些系统采用截断或填充等方式处理，以保持一致的向量表示。

对于自然语言，可以推荐使用Token分割器，结合Chunk Size和Overlap Size可以得到不同的切分：

- Chunk Size（块大小）：表示将文本划分为较小块的大小。这是分割后每个独立文本块的长度或容量。块大小的选择取决于应用的需求和对文本结构的理解
- Overlap Size（重叠大小）：指相邻两个文本块之间的重叠部分的大小。在切割文本时，通常希望保留一些上下文信息，重叠大小就是控制这种上下文保留的参数。
以下是不同的分割器的类型


**文本分割器的评价**

可以通过  [chunkviz utility](https://chunkviz.up.railway.app/) 来对分割器进行评价

![文本分割器](img/text_spliter.png)


#### 文本多路召回与重排序
目的：实现多种文本编码和检索逻辑，并进行重排序

##### 多路召回逻辑
多路召回逻辑是在文本检索中常用的一种策略，其目的是通过多个召回路径（或方法）综合获取候选文档，以提高检索的全面性和准确性。单一的召回方法可能由于模型特性或数据特点而存在局限性，多路召回逻辑引入了多个召回路径，每个路径采用不同的召回方法。
- 实现方法1：将BM25的检索结果 和 语义检索结果 按照排名进行加权
- 实现方法2：按照段落、句子、页不同的角度进行语义编码进行检索，综合得到检索结果。
![多路召回](img/fusion.png)

##### 重排序逻辑（BM25 + BGE Rerank）
重排序逻辑是文本检索领域中一种重要的策略，主要用于优化原有文本检索方法返回的候选文档顺序，以提高最终的检索效果。在传统的文本检索方法中，往往采用打分的逻辑，如计算BERT嵌入向量之间的相似度。而重排序逻辑引入了更为复杂的文本交叉方法，通过特征交叉得到更进一步的打分，从而提高排序的准确性。
- 重排序逻辑常常使用更为强大的模型，如交叉编码器（cross-encoder）模型。这类模型能够更好地理解文本之间的交叉关系，捕捉更复杂的语义信息。
- 首先通过传统的嵌入模型获取初始的Top-k文档，然后使用重排序逻辑对这些文档进行重新排序。这样可以在保留初步筛选文档的基础上，更精确地排列它们的顺序。

![重排序](img/rerank.png)

In [6]:
import jieba, json, pdfplumber
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from rank_bm25 import BM25Okapi

In [7]:
## load data
questions = json.load(open("questions.json"))
pdf = pdfplumber.open("trainData.pdf")
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })

In [36]:
# 加载重排序模型
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-base')
rerank_model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-base')
rerank_model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [39]:
pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]
bm25 = BM25Okapi(pdf_content_words)

# 首先进行BM25检索 计算相关度
for query_idx in range(len(questions)):
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    
    # 选取top3
    max_score_page_idxs = doc_scores.argsort()[-3:]
    pairs = []
    for idx in max_score_page_idxs:
        pairs.append([questions[query_idx]["question"], pdf_content[idx]['content']])
    
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors="pt", max_length=512)

    with torch.no_grad():
        inputs = {key: inputs[key] for key in inputs.keys()}
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
    
    max_score_page_idx = max_score_page_idxs[scores.numpy().argmax()]
    questions[query_idx]['reference'] = "page_" + str(max_score_page_idx + 1)

In [40]:
with open('submitLM.json', 'w', encoding='utf-8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

In [9]:
import time
import jwt
import requests

def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }

    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

In [2]:
import os
ZP_key = os.getenv("GLM_KEY")

In [3]:
def ask_glm(content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token(ZP_key, 1000)
    }

    data = {
        "model": "glm-3-turbo",
        "messages": [{"role": "user", "content": content}]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [4]:
import json

with open("submitLM.json", "r", encoding='utf-8') as f:
    info = json.load(f)


In [10]:
# 运行时解除注释
# for query_idx in range(len(questions)):
#     target_index = int(info[query_idx]['reference'].split("_")[1])
    
#     prompt = '''你是一个汽车专家，帮我结合给定的资料，回答一个问题。如果问题无法从资料中获得，请输出结合给定的资料，无法回答问题。
# 资料：{0}

# 问题：{1}
#     '''.format(pdf_content[target_index-1]['content'],
#                questions[query_idx]['question']
#                )

#     answer = ask_glm(prompt)['choices'][0]['message']['content']
#     questions[query_idx]['answer'] = answer


# with open('submitM3e.json', "w", encoding='utf-8') as up:
#     json.dump(questions, up, ensure_ascii=False, indent=4)
# # 这一版的分数是在0.3185 还可以继续优化

InvalidSchema: Missing dependencies for SOCKS support.

In [15]:
count = 0
not_list = []
for query_idx in range(len(questions)):
    target_index = int(info[query_idx]['reference'].split("_")[1])
    
    prompt = '''你是一个汽车维修和汽车销售的专家,请判断下面的提问是否与汽车使用或汽车使用手册的内容有关。
    回答的输出形式是 相关 或者 不相关。
    问题: {}
'''.format(questions[query_idx]['question'])
#     prompt = '''你是一个汽车专家，帮我结合给定的资料，回答一个问题。如果问题无法从资料中获得，请输出结合给定的资料，无法回答问题。
# 资料：{0}

# 问题：{1}
#     '''.format(pdf_content[target_index-1]['content'],
#                questions[query_idx]['question']
#                )

    answer = ask_glm(prompt)['choices'][0]['message']['content']
    if answer == "不相关":
        count += 1
        not_list.append(query_idx)
    # questions[query_idx]['answer'] = answer

"""
在实际的submit.json文件中 只有28个问题无法回答，但是这个不相关的问题的长度通过提示词回答是98，
这显然不能让他自己发挥，
检查了这28个大语言模型无法回答的问题，
其实是与汽车相关的，更多的可能是并没有匹配到的问题，是有少部分问的问题比较算常识吧
另外，有些问题的关键词出现在了回答中，但是并没有相应的下一步的细节，
可以推测，可能是粗暴的将每一个pdf的每一页试做一个文档导致的，没有连贯起来，
因此合并某些文档的策略来帮助生成回答是有效的。
或者一些很简单的策略便是，给回答的文档的区间-+1 前进一页再增加一页
而目前没有与汽车无法的问题，
"""

301

In [ ]:
prompt_normal = "你是一个汽车维修和汽车销售的专家, 请帮我回答以下汽车使用和维护的相关问题"